## St Lucia Cement Pred



In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
import geopandas as gpd
from resizeimage import resizeimage
import datetime
import uuid
from os import listdir
from os.path import isfile, join

In [3]:
torch.cuda.set_device(0)
torch.cuda.current_device()

0

In [4]:
data_dir = Path('data')
sub_dir=data_dir/'submissions'
st_lucia_castries = Path('data/stac/st_lucia/castries')
COUNTRY='st_lucia'
REGION='castries'
DATASET_PATH=st_lucia_castries
TRAIN_JSON = f'train-{REGION}.geojson'

In [5]:
img_size=256

In [6]:
train_images=data_dir/f'train/rotated/clipped/reflect/{img_size}'
test_images=data_dir/f'test/rotated/clipped/reflect/{img_size}'

In [7]:
df_st_cc=pd.read_csv(sub_dir/'stage-2-rn50-03_11-cv_reflect_101_valid-20191109-123cca5f.csv')

In [8]:
df_st_cc.head()

,id,concrete_cement,healthy_metal,incomplete,irregular_metal,other
0,7a4d630a,0.000964,0.991899,0.001250,0.003872,0.002015
1,7a4bbbd6,0.002175,0.991627,0.000803,0.004684,0.000711
2,7a4ac744,0.008024,0.417557,0.010922,0.561353,0.002144
3,7a4881fa,0.001661,0.018901,0.010896,0.968236,0.000307
4,7a4aa4a8,0.004693,0.853668,0.002853,0.135639,0.003147


In [9]:
df_st_cc_preds=df_st_cc.drop(columns=['id'])

In [10]:
pred_classes=df_st_cc_preds.idxmax(axis=1)

In [11]:
df_st_cc['pred']=pred_classes

In [12]:
df_st_cc.head()

,id,concrete_cement,healthy_metal,incomplete,irregular_metal,other,pred
0,7a4d630a,0.000964,0.991899,0.001250,0.003872,0.002015,healthy_metal
1,7a4bbbd6,0.002175,0.991627,0.000803,0.004684,0.000711,healthy_metal
2,7a4ac744,0.008024,0.417557,0.010922,0.561353,0.002144,irregular_metal
3,7a4881fa,0.001661,0.018901,0.010896,0.968236,0.000307,irregular_metal
4,7a4aa4a8,0.004693,0.853668,0.002853,0.135639,0.003147,healthy_metal


In [13]:
label_df = gpd.read_file(DATASET_PATH/TRAIN_JSON)
label_df = label_df[label_df['geometry'].isna() != True] # remove empty rows

In [14]:
label_df.head()

,id,roof_material,verified,geometry
0,7a24bd7e,healthy_metal,False,"POLYGON ((-60.99638 13.99362, -60.99626 13.993..."
1,7a2cf00c,healthy_metal,False,"POLYGON ((-60.99657 13.99379, -60.99662 13.993..."
2,7a3975e8,healthy_metal,False,"POLYGON ((-60.99189 13.99392, -60.99179 13.993..."
3,7a1d215e,healthy_metal,False,"POLYGON ((-60.99345 13.99396, -60.99330 13.993..."
4,7a42475e,healthy_metal,False,"POLYGON ((-60.99617 13.99408, -60.99609 13.994..."


In [15]:
label_cc=label_df.loc[label_df['roof_material'] ==  'concrete_cement']

In [16]:
label_cc.head()

,id,roof_material,verified,geometry
20,7a23d864,concrete_cement,False,"POLYGON ((-60.99474 13.99438, -60.99474 13.994..."
74,7a297f76,concrete_cement,False,"POLYGON ((-60.99817 13.99509, -60.99802 13.995..."
198,7a34ef78,concrete_cement,False,"POLYGON ((-60.99777 13.99632, -60.99783 13.996..."
216,7a3dc49a,concrete_cement,False,"POLYGON ((-60.99531 13.99688, -60.99535 13.996..."
287,7a26dfa0,concrete_cement,False,"POLYGON ((-60.98675 13.99786, -60.98669 13.997..."


In [17]:
len(label_cc)

89

In [18]:
unver_cc = label_cc.id.tolist()
unver_cc_roof = label_cc.roof_material.tolist()
st_cc=df_st_cc.id.tolist()

In [19]:
unver_cc[:10]

['7a23d864',
 '7a297f76',
 '7a34ef78',
 '7a3dc49a',
 '7a26dfa0',
 '7a33c346',
 '7a3e0126',
 '7a30b9d0',
 '7a2f2cf0',
 '7a27cda2']

In [20]:
df_st_cc.head()

,id,concrete_cement,healthy_metal,incomplete,irregular_metal,other,pred
0,7a4d630a,0.000964,0.991899,0.001250,0.003872,0.002015,healthy_metal
1,7a4bbbd6,0.002175,0.991627,0.000803,0.004684,0.000711,healthy_metal
2,7a4ac744,0.008024,0.417557,0.010922,0.561353,0.002144,irregular_metal
3,7a4881fa,0.001661,0.018901,0.010896,0.968236,0.000307,irregular_metal
4,7a4aa4a8,0.004693,0.853668,0.002853,0.135639,0.003147,healthy_metal


In [21]:
### This needs QC!!!

In [22]:
df_npu=pd.read_csv(data_dir/'non_predicted_unverfied.csv')

In [23]:
df_npu.head()

,id,roof_material
0,7a295212,healthy_metal
1,7a3e455a,other
2,7a26dfa0,concrete_cement
3,7a44cd9e,other
4,7a3e5482,other


In [24]:
bad_ids=df_npu.id.values.tolist()

In [25]:
df_all = pd.read_csv(data_dir/'df_train_all.csv')

In [26]:
df_uv=df_all.loc[df_all['verified'] == False]

In [27]:
df_pred_v=df_uv.loc[~df_uv['id'].isin(bad_ids)]

In [28]:
len(df_pred_v)

7556

In [29]:
df_pred_v.head()

,id,roof_material,verified,country,region
13866,7a24bd7e,healthy_metal,False,st_lucia,castries
13867,7a2cf00c,healthy_metal,False,st_lucia,castries
13868,7a3975e8,healthy_metal,False,st_lucia,castries
13869,7a1d215e,healthy_metal,False,st_lucia,castries
13870,7a42475e,healthy_metal,False,st_lucia,castries


In [30]:
df_pred_v_cc=df_pred_v.loc[df_pred_v['roof_material'] == 'concrete_cement']

In [31]:
len(df_pred_v_cc)

105

In [32]:
b1=label_cc.id.tolist()
b2=df_pred_v_cc.id.tolist()

The gold ids - TODO - double check this

In [33]:
len(set(b1).intersection(b2))

75

In [34]:
gold_ids = list(set(b1).intersection(b2))


In [35]:
df=pd.DataFrame({"id":gold_ids})
df.head()

,id
0,7a1dd158
1,7a32aa7e
2,7a3de768
3,7a2087b8
4,7a3cc4fa


In [36]:

df.to_csv(data_dir/'st_lucia_castries_gold_concrete_cement.csv', index=False)